# Iteration 3: Sample length of stay on the ASU stage 2

## Plain English Summary

In the model length of stay is determined not just by patient type, but also patient destination post ASU.  For example Early Supported Discharge greatly reduces length of stay.  Now that we have sampled the destination we can sample the length of stay from Lognormal distributions.



## LLM Prompt

::::{admonition} LLM Prompt
:::{include} ../04_prompts_stage1/inputs/03_iteration.md
:::
::::

## Date and time of input to LLM

> 1130 28th June 2024

## Observations on the LLM Output

* The new code looks fine and works. However tests need to be modified each time to accept the additional parameters.
* An additional prompt is provided to allow running the model with partial parameterisation specification for scenarios (and tests).
* A new function is created in class `PatientType` converting parameters to normal parameters. It looks correct, but will be checked in the next iteration.


## Sources reported by the LLM

* None given - Perplexity appears to have stopped providing them.

## LLM Output

### Response:

Updated PatientType class to include los_params and a method to sample length of stay.
Modified ASUModel initialization to include length of stay parameters.
Added acute_treatment method to ASUModel which calls the appropriate treatment method based on patient type.
Implemented specific acute treatment methods for each patient type (stroke_acute_treatment, tia_acute_treatment, complex_neurological_acute_treatment, other_acute_treatment).
Updated patient_generator to call acute_treatment as a SimPy process.
Each acute treatment method samples the length of stay, yields a timeout, and prints relevant information.

This modified model now includes the acute treatment process for each patient type, with length of stay sampled from the specified lognormal distributions. The model prints out useful information after each event, including patient arrival, start of acute treatment, and end of acute treatment. To run the simulation with custom parameters, you can modify the main() function call as needed. The current setup uses the default parameters specified in the problem statement.

In [4]:
import simpy
import numpy as np
import math

class PatientType:
    def __init__(self, name, interarrival_time, post_asu_probabilities, los_params):
        self.name = name
        self.interarrival_time = interarrival_time
        self.count = 0
        self.rng = np.random.default_rng()
        self.post_asu_probabilities = post_asu_probabilities
        self.los_params = los_params

    def generate_interarrival_time(self):
        return self.rng.exponential(self.interarrival_time)

    def sample_post_asu_destination(self):
        return self.rng.choice(['Rehab', 'ESD', 'Other'], p=self.post_asu_probabilities)

    def sample_length_of_stay(self, destination=None):
        if self.name == 'Stroke':
            mean, std = self.los_params[destination]
        else:
            mean, std = self.los_params

        # Convert lognormal parameters
        normal_mean = math.log(mean**2 / math.sqrt(std**2 + mean**2))
        normal_std = math.sqrt(math.log(1 + (std**2 / mean**2)))

        return self.rng.lognormal(normal_mean, normal_std)

class ASUModel:
    def __init__(self, env, patient_params=None, run_length=5 * 365):
        self.env = env
        self.run_length = run_length
        self.total_arrivals = 0
        
        if patient_params is None:
            self.patient_types = {
                'Stroke': PatientType('Stroke', 1.2, [0.24, 0.13, 0.63], 
                                      {'Rehab': (7.4, 8.6), 'ESD': (4.6, 4.8), 'Other': (7.0, 8.7)}),
                'TIA': PatientType('TIA', 9.3, [0.01, 0.01, 0.98], (1.8, 5.0)),
                'Complex Neurological': PatientType('Complex Neurological', 3.6, [0.11, 0.05, 0.84], (4.0, 5.0)),
                'Other': PatientType('Other', 3.2, [0.05, 0.10, 0.85], (3.8, 5.2))
            }
        else:
            self.patient_types = {
                name: PatientType(name, params['interarrival_time'], params['post_asu_probabilities'], params['los_params'])
                for name, params in patient_params.items()
            }

    def run(self):
        for patient_type in self.patient_types.values():
            self.env.process(self.patient_generator(patient_type))
        self.env.run(until=self.run_length)

    def patient_generator(self, patient_type):
        while True:
            interarrival_time = patient_type.generate_interarrival_time()
            yield self.env.timeout(interarrival_time)
            
            self.total_arrivals += 1
            patient_type.count += 1
            patient_id = self.total_arrivals - 1

            post_asu_destination = patient_type.sample_post_asu_destination()

            print(f"Time {self.env.now:.2f}: Patient {patient_id} ({patient_type.name}) arrived")
            print(f"  Total arrivals: {self.total_arrivals}")
            print(f"  {patient_type.name} arrivals: {patient_type.count}")
            print(f"  Post-ASU destination: {post_asu_destination}")
            print(f"  Next {patient_type.name} arrival in {interarrival_time:.2f} days")

            # Call acute treatment process
            self.env.process(self.acute_treatment(patient_type, patient_id, post_asu_destination))

    def acute_treatment(self, patient_type, patient_id, post_asu_destination):
        if patient_type.name == 'Stroke':
            yield from self.stroke_acute_treatment(patient_type, patient_id, post_asu_destination)
        elif patient_type.name == 'TIA':
            yield from self.tia_acute_treatment(patient_type, patient_id)
        elif patient_type.name == 'Complex Neurological':
            yield from self.complex_neurological_acute_treatment(patient_type, patient_id)
        else:  # Other
            yield from self.other_acute_treatment(patient_type, patient_id)

    def stroke_acute_treatment(self, patient_type, patient_id, post_asu_destination):
        los = patient_type.sample_length_of_stay(post_asu_destination)
        print(f"Time {self.env.now:.2f}: Patient {patient_id} (Stroke) starting acute treatment")
        print(f"  Length of stay: {los:.2f} days")
        yield self.env.timeout(los)
        print(f"Time {self.env.now:.2f}: Patient {patient_id} (Stroke) finished acute treatment")

    def tia_acute_treatment(self, patient_type, patient_id):
        los = patient_type.sample_length_of_stay()
        print(f"Time {self.env.now:.2f}: Patient {patient_id} (TIA) starting acute treatment")
        print(f"  Length of stay: {los:.2f} days")
        yield self.env.timeout(los)
        print(f"Time {self.env.now:.2f}: Patient {patient_id} (TIA) finished acute treatment")

    def complex_neurological_acute_treatment(self, patient_type, patient_id):
        los = patient_type.sample_length_of_stay()
        print(f"Time {self.env.now:.2f}: Patient {patient_id} (Complex Neurological) starting acute treatment")
        print(f"  Length of stay: {los:.2f} days")
        yield self.env.timeout(los)
        print(f"Time {self.env.now:.2f}: Patient {patient_id} (Complex Neurological) finished acute treatment")

    def other_acute_treatment(self, patient_type, patient_id):
        los = patient_type.sample_length_of_stay()
        print(f"Time {self.env.now:.2f}: Patient {patient_id} (Other) starting acute treatment")
        print(f"  Length of stay: {los:.2f} days")
        yield self.env.timeout(los)
        print(f"Time {self.env.now:.2f}: Patient {patient_id} (Other) finished acute treatment")

def main(patient_params=None, run_length=5*365):
    env = simpy.Environment()
    model = ASUModel(env, patient_params, run_length)
    model.run()

    print("\nSimulation completed.")
    print(f"Total simulation time: {model.env.now:.2f} days")
    print(f"Total patient arrivals: {model.total_arrivals}")
    for patient_type in model.patient_types.values():
        print(f"Total {patient_type.name} arrivals: {patient_type.count}")

if __name__ == "__main__":
    main()


Time 0.91: Patient 0 (Stroke) arrived
  Total arrivals: 1
  Stroke arrivals: 1
  Post-ASU destination: ESD
  Next Stroke arrival in 0.91 days
Time 0.91: Patient 0 (Stroke) starting acute treatment
  Length of stay: 2.72 days
Time 2.69: Patient 1 (Stroke) arrived
  Total arrivals: 2
  Stroke arrivals: 2
  Post-ASU destination: Other
  Next Stroke arrival in 1.77 days
Time 2.69: Patient 1 (Stroke) starting acute treatment
  Length of stay: 2.15 days
Time 2.70: Patient 2 (Other) arrived
  Total arrivals: 3
  Other arrivals: 1
  Post-ASU destination: Other
  Next Other arrival in 2.70 days
Time 2.70: Patient 2 (Other) starting acute treatment
  Length of stay: 2.17 days
Time 2.86: Patient 3 (Stroke) arrived
  Total arrivals: 4
  Stroke arrivals: 3
  Post-ASU destination: Rehab
  Next Stroke arrival in 0.18 days
Time 2.86: Patient 3 (Stroke) starting acute treatment
  Length of stay: 2.26 days
Time 3.01: Patient 4 (Stroke) arrived
  Total arrivals: 5
  Stroke arrivals: 4
  Post-ASU destinat

## Additional prompt: 
modify the `__init__` method of `ASUModel` to handle partial parameter specifications.

In [11]:
import simpy
import numpy as np
import math

class PatientType:
    def __init__(self, name, interarrival_time, post_asu_probabilities, los_params):
        self.name = name
        self.interarrival_time = interarrival_time
        self.count = 0
        self.rng = np.random.default_rng()
        self.post_asu_probabilities = post_asu_probabilities
        self.los_params = los_params

    def generate_interarrival_time(self):
        return self.rng.exponential(self.interarrival_time)

    def sample_post_asu_destination(self):
        return self.rng.choice(['Rehab', 'ESD', 'Other'], p=self.post_asu_probabilities)

    def sample_length_of_stay(self, destination=None):
        if self.name == 'Stroke':
            mean, std = self.los_params[destination]
        else:
            mean, std = self.los_params

        # Convert lognormal parameters
        normal_mean = math.log(mean**2 / math.sqrt(std**2 + mean**2))
        normal_std = math.sqrt(math.log(1 + (std**2 / mean**2)))

        return self.rng.lognormal(normal_mean, normal_std)

class ASUModel:
    def __init__(self, env, patient_params=None, run_length=5 * 365):
        self.env = env
        self.run_length = run_length
        self.total_arrivals = 0
        
        default_params = {
            'Stroke': {
                'interarrival_time': 1.2,
                'post_asu_probabilities': [0.24, 0.13, 0.63],
                'los_params': {'Rehab': (7.4, 8.6), 'ESD': (4.6, 4.8), 'Other': (7.0, 8.7)}
            },
            'TIA': {
                'interarrival_time': 9.3,
                'post_asu_probabilities': [0.01, 0.01, 0.98],
                'los_params': (1.8, 5.0)
            },
            'Complex Neurological': {
                'interarrival_time': 3.6,
                'post_asu_probabilities': [0.11, 0.05, 0.84],
                'los_params': (4.0, 5.0)
            },
            'Other': {
                'interarrival_time': 3.2,
                'post_asu_probabilities': [0.05, 0.10, 0.85],
                'los_params': (3.8, 5.2)
            }
        }
        
        if patient_params is None:
            patient_params = default_params
        else:
            # Merge provided params with defaults
            for patient_type, params in default_params.items():
                if patient_type in patient_params:
                    params.update(patient_params[patient_type])
                patient_params[patient_type] = params

        self.patient_types = {
            name: PatientType(name, 
                              params['interarrival_time'], 
                              params['post_asu_probabilities'], 
                              params['los_params'])
            for name, params in patient_params.items()
        }

    def run(self):
        for patient_type in self.patient_types.values():
            self.env.process(self.patient_generator(patient_type))
        self.env.run(until=self.run_length)

    def patient_generator(self, patient_type):
        while True:
            interarrival_time = patient_type.generate_interarrival_time()
            yield self.env.timeout(interarrival_time)
            
            self.total_arrivals += 1
            patient_type.count += 1
            patient_id = self.total_arrivals - 1

            post_asu_destination = patient_type.sample_post_asu_destination()

            print(f"Time {self.env.now:.2f}: Patient {patient_id} ({patient_type.name}) arrived")
            print(f"  Total arrivals: {self.total_arrivals}")
            print(f"  {patient_type.name} arrivals: {patient_type.count}")
            print(f"  Post-ASU destination: {post_asu_destination}")
            print(f"  Next {patient_type.name} arrival in {interarrival_time:.2f} days")

            # Call acute treatment process
            self.env.process(self.acute_treatment(patient_type, patient_id, post_asu_destination))

    def acute_treatment(self, patient_type, patient_id, post_asu_destination):
        if patient_type.name == 'Stroke':
            yield from self.stroke_acute_treatment(patient_type, patient_id, post_asu_destination)
        elif patient_type.name == 'TIA':
            yield from self.tia_acute_treatment(patient_type, patient_id)
        elif patient_type.name == 'Complex Neurological':
            yield from self.complex_neurological_acute_treatment(patient_type, patient_id)
        else:  # Other
            yield from self.other_acute_treatment(patient_type, patient_id)

    def stroke_acute_treatment(self, patient_type, patient_id, post_asu_destination):
        los = patient_type.sample_length_of_stay(post_asu_destination)
        print(f"Time {self.env.now:.2f}: Patient {patient_id} (Stroke) starting acute treatment")
        print(f"  Length of stay: {los:.2f} days")
        yield self.env.timeout(los)
        print(f"Time {self.env.now:.2f}: Patient {patient_id} (Stroke) finished acute treatment")

    def tia_acute_treatment(self, patient_type, patient_id):
        los = patient_type.sample_length_of_stay()
        print(f"Time {self.env.now:.2f}: Patient {patient_id} (TIA) starting acute treatment")
        print(f"  Length of stay: {los:.2f} days")
        yield self.env.timeout(los)
        print(f"Time {self.env.now:.2f}: Patient {patient_id} (TIA) finished acute treatment")

    def complex_neurological_acute_treatment(self, patient_type, patient_id):
        los = patient_type.sample_length_of_stay()
        print(f"Time {self.env.now:.2f}: Patient {patient_id} (Complex Neurological) starting acute treatment")
        print(f"  Length of stay: {los:.2f} days")
        yield self.env.timeout(los)
        print(f"Time {self.env.now:.2f}: Patient {patient_id} (Complex Neurological) finished acute treatment")

    def other_acute_treatment(self, patient_type, patient_id):
        los = patient_type.sample_length_of_stay()
        print(f"Time {self.env.now:.2f}: Patient {patient_id} (Other) starting acute treatment")
        print(f"  Length of stay: {los:.2f} days")
        yield self.env.timeout(los)
        print(f"Time {self.env.now:.2f}: Patient {patient_id} (Other) finished acute treatment")

def main(patient_params=None, run_length=5*365):
    env = simpy.Environment()
    model = ASUModel(env, patient_params, run_length)
    model.run()

    print("\nSimulation completed.")
    print(f"Total simulation time: {model.env.now:.2f} days")
    print(f"Total patient arrivals: {model.total_arrivals}")
    for patient_type in model.patient_types.values():
        print(f"Total {patient_type.name} arrivals: {patient_type.count}")

if __name__ == "__main__":
    main()


Time 0.02: Patient 0 (Stroke) arrived
  Total arrivals: 1
  Stroke arrivals: 1
  Post-ASU destination: Rehab
  Next Stroke arrival in 0.02 days
Time 0.02: Patient 0 (Stroke) starting acute treatment
  Length of stay: 4.79 days
Time 1.06: Patient 1 (Stroke) arrived
  Total arrivals: 2
  Stroke arrivals: 2
  Post-ASU destination: Other
  Next Stroke arrival in 1.04 days
Time 1.06: Patient 1 (Stroke) starting acute treatment
  Length of stay: 1.88 days
Time 2.22: Patient 2 (Complex Neurological) arrived
  Total arrivals: 3
  Complex Neurological arrivals: 1
  Post-ASU destination: Other
  Next Complex Neurological arrival in 2.22 days
Time 2.22: Patient 2 (Complex Neurological) starting acute treatment
  Length of stay: 6.38 days
Time 2.94: Patient 1 (Stroke) finished acute treatment
Time 3.68: Patient 3 (Complex Neurological) arrived
  Total arrivals: 4
  Complex Neurological arrivals: 2
  Post-ASU destination: Other
  Next Complex Neurological arrival in 1.46 days
Time 3.68: Patient 3 (

## Testing

* **Extreme value test 1**: TIA, Complex Neuro, Other, have their inter-arrival time is set to $M$ a very large number
    * Expected result: The only type of patient to arrive to the model is "Stroke"
    * Actual result (PASS): The only type of patient to arrive to the model is "Accident and Emergency."
* **Extreme value test 2**: All patient types have have their inter-arrival time is set to $M$ a very large number
    * Expected result: No patients arrive to the model.
    * Actual result (PASS): No patients arrive to the model.
* **Different run length**: The run length of the model is set to 10 days.
    * Expected result: The model should runs no longer than 240 simulated time periods
    * Actual result (PASS): The model runs no longer than 240 simulated time periods




### Extreme value test 1

The model uses hard coded parameters.  The most simple way to modify the code for this test was to change hard coded parameters for $M$

**Result: PASS**

In [8]:
M = 10_000_000

In [12]:
def test_1(large_number):
    # Create the simulation environment
    if __name__ == "__main__":
        asu_arrival_rates = {
            'Stroke': {'interarrival_time': 1.2},
            'TIA': {'interarrival_time': large_number},
            'Complex Neurological': {'interarrival_time': large_number},
            'Other': {'interarrival_time': large_number}
        }
        main(asu_arrival_rates, run_length=2*365)  # Run for 2 years with custom parameters


In [13]:
test_1(M)

Time 1.32: Patient 0 (Stroke) arrived
  Total arrivals: 1
  Stroke arrivals: 1
  Post-ASU destination: Rehab
  Next Stroke arrival in 1.32 days
Time 1.32: Patient 0 (Stroke) starting acute treatment
  Length of stay: 14.72 days
Time 1.99: Patient 1 (Stroke) arrived
  Total arrivals: 2
  Stroke arrivals: 2
  Post-ASU destination: ESD
  Next Stroke arrival in 0.67 days
Time 1.99: Patient 1 (Stroke) starting acute treatment
  Length of stay: 1.43 days
Time 2.44: Patient 2 (Stroke) arrived
  Total arrivals: 3
  Stroke arrivals: 3
  Post-ASU destination: Other
  Next Stroke arrival in 0.45 days
Time 2.44: Patient 2 (Stroke) starting acute treatment
  Length of stay: 1.42 days
Time 2.51: Patient 3 (Stroke) arrived
  Total arrivals: 4
  Stroke arrivals: 4
  Post-ASU destination: Other
  Next Stroke arrival in 0.07 days
Time 2.51: Patient 3 (Stroke) starting acute treatment
  Length of stay: 2.84 days
Time 3.42: Patient 1 (Stroke) finished acute treatment
Time 3.86: Patient 2 (Stroke) finished

### Extreme value test 2

* The code is again modified so that all patient typs have a mean inter-arrival rate of $M$

**Result: PASS**

In [14]:
def test_2(large_number):
    # Create the simulation environment
    if __name__ == "__main__":
        asu_arrival_rates = {
            'Stroke': {'interarrival_time': large_number},
            'TIA': {'interarrival_time': large_number},
            'Complex Neurological': {'interarrival_time': large_number},
            'Other': {'interarrival_time': large_number}
        }
        main(asu_arrival_rates, run_length=2*365)  # Run for 2 years with custom parameters


In [15]:
test_2(M)


Simulation completed.
Total simulation time: 730.00 days
Total patient arrivals: 0
Total Stroke arrivals: 0
Total TIA arrivals: 0
Total Complex Neurological arrivals: 0
Total Other arrivals: 0


### Different run length test

The model has again been modified to use the original code and parameters generated by the LLM.

In [18]:
TEN_DAYS = 10

In [19]:
def test_3(run_length):
    # Create the simulation environment
    if __name__ == "__main__":
        main(run_length=run_length)  # Run for 2 years with custom parameters


In [20]:
test_3(TEN_DAYS)

Time 0.92: Patient 0 (Stroke) arrived
  Total arrivals: 1
  Stroke arrivals: 1
  Post-ASU destination: Other
  Next Stroke arrival in 0.92 days
Time 0.92: Patient 0 (Stroke) starting acute treatment
  Length of stay: 6.30 days
Time 1.02: Patient 1 (Other) arrived
  Total arrivals: 2
  Other arrivals: 1
  Post-ASU destination: ESD
  Next Other arrival in 1.02 days
Time 1.02: Patient 1 (Other) starting acute treatment
  Length of stay: 1.98 days
Time 1.12: Patient 2 (Complex Neurological) arrived
  Total arrivals: 3
  Complex Neurological arrivals: 1
  Post-ASU destination: Other
  Next Complex Neurological arrival in 1.12 days
Time 1.12: Patient 2 (Complex Neurological) starting acute treatment
  Length of stay: 2.50 days
Time 1.59: Patient 3 (Complex Neurological) arrived
  Total arrivals: 4
  Complex Neurological arrivals: 2
  Post-ASU destination: Other
  Next Complex Neurological arrival in 0.47 days
Time 1.59: Patient 3 (Complex Neurological) starting acute treatment
  Length of st